In [3]:
class myconfig():
    data_path = r"/home/work/lib_data/KISTI/wiki_summary_data.csv"
    sample_data_path = r"/home/work/lib_data/KISTI/0708_sample_data.pkl"
mcfg = myconfig()

In [2]:
import ollama
ollama_model = "llama3:70b-instruct"

# sample data 만들기

In [3]:
import pandas as pd
data = pd.read_csv(mcfg.data_path)

In [4]:
data

,title,summary
0,Kigali Declaration on Neglected Tropical Diseases,The Kigali Declaration on Neglected Tropical D...
1,List of former Protestants,Former Protestants or ex-Protestants are peopl...
2,Audianism,"Audianism, or Anthropomorphism, was a sect of ..."
3,List of New Zealand exchange-traded funds,An exchange-traded fund (ETF) is an investment...
4,Arsenic poisoning,Arsenic poisoning (or arsenicosis) is a medica...
...,...,...
53512,Computational topology,"Algorithmic topology, or computational topolog..."
53513,Waste minimisation,Waste minimisation is a set of processes and p...
53514,Chief technology officer,A chief technology officer (CTO) (also known a...
53515,Mobile commerce,The term mobile commerce was originally coined...


In [5]:
sample_data = pd.DataFrame(data.sample(n=500, random_state=100))
sample_data.reset_index(drop=True, inplace=True)

In [6]:
sample_data.to_pickle(mcfg.sample_data_path)

1차 라벨링 시작

In [5]:
import pandas as pd
sample_data = pd.read_pickle(mcfg.sample_data_path)

In [6]:
sample_data

,title,summary
0,River delta,A river delta is a landform shaped like a tria...
1,Chinese ophthalmology,Chinese ophthalmology (Chinese: 中医眼科学; pinyin:...
2,Error-correcting codes with feedback,"In mathematics, computer science, telecommunic..."
3,Localizer performance with vertical guidance,Localizer performance with vertical guidance (...
4,AK Press,"AK Press is a worker-managed, independent publ..."
...,...,...
495,Human resource management system,A human resources management system (HRMS) or ...
496,Central bank,"A central bank, reserve bank, national bank, o..."
497,Hydraulic engineering,Hydraulic engineering as a sub-discipline of c...
498,Director (military),"A director, also called an auxiliary predictor..."


In [7]:
title = list(sample_data["title"])
summary = list(sample_data["summary"])

In [8]:
sample_response = []

In [10]:
for i in range(len(title)):
    prompt = f"""Classify 'Item' or 'Non-item'.
    Category : Item 
    Item is a technology, an idea that can have commercial, national, or social value in the commercial market. It also refers to a product and a service that is currently commercialized in the market or the technology to implement it. 
       Sub-categories of 'Item' : technologies, products, services, ideas, and raw materials.

    Category : Non-item
       Sub-categories of 'Non-item' :person name, places, organizations, certifications, regulations, laws...
  
    Here's a short description of '{title[i]}':
    '{summary[i]}'
  
    According to the description, what is '{title[i]}' categorized as?

    Please answer in the following form
    1. Category(Item/Non-item): 
    2. Sub-category of Category: 
    3. Explanation: 
    """
    
    
    
    response = ollama.chat(
    model=ollama_model,
    messages=[
        {"role": "system", "content": "You are a helpful chatbot which is expertised in business."},
        {'role': 'user', 'content': prompt}])
    
    print(response['message']['content'])
    sample_response.append(response['message']['content'])

Here's the categorization of 'River delta':

1. Category: Non-item
2. Sub-category of Category: places
3. Explanation: The description of 'River delta' refers to a landform, specifically a geographical location where sediment is deposited by a river, forming a triangular shape. It does not fit into any of the sub-categories under 'Item', such as technologies, products, services, ideas, or raw materials. Instead, it falls under the sub-category of 'places' which is a type of Non-item.
Here is the classification:

1. Category: Item
2. Sub-category of Category: Technology/Idea
3. Explanation: Chinese ophthalmology is a technology or an idea that has commercial, national, or social value in the medical market. It refers to a specific approach to treating eye diseases using traditional Chinese medicine methods, including acupuncture, herbal remedies, and qigong exercises. As it is a distinct approach to healthcare with its own theories, practices, and techniques, it can be considered an ite

In [11]:
len(sample_response)

500

In [12]:
def extract_data(text):
    # Split the text data into lines
    lines = text.strip().split('\n')
    
    # Initialize an empty dictionary to store the data
    data = {}

    # Loop through each line and extract key-value pairs
    for line in lines:
        if ': ' in line:
            key, value = line.split(': ', 1)  # Split only on the first ': ' to handle cases with ':' in value
            # Convert numerical values to float where applicable
            try:
                value = float(value)
            except ValueError:
                pass
            data[key] = value
    return data

In [13]:
seperate_data= [extract_data(text) for text in sample_response]
df_combined = pd.DataFrame(seperate_data)

In [14]:
df_final = pd.concat([sample_data,df_combined],axis=1)

In [16]:
df_final.to_excel("./라벨링할 것.xlsx",index=False)

In [15]:
df_final

,title,summary,1. Category,2. Sub-category of Category,3. Explanation,1. Category(Item/Non-item)
0,River delta,A river delta is a landform shaped like a tria...,Non-item,places,The description of 'River delta' refers to a l...,NaN
1,Chinese ophthalmology,Chinese ophthalmology (Chinese: 中医眼科学; pinyin:...,Item,Technology/Idea,Chinese ophthalmology is a technology or an id...,NaN
2,Error-correcting codes with feedback,"In mathematics, computer science, telecommunic...",Item,Technology/Idea,The description of 'Error-correcting codes wit...,NaN
3,Localizer performance with vertical guidance,Localizer performance with vertical guidance (...,Item,Technology/Service,The description highlights that LPV is a techn...,NaN
4,AK Press,"AK Press is a worker-managed, independent publ...",Non-item,Organization,"According to the description, AK Press is a co...",NaN
...,...,...,...,...,...,...
495,Human resource management system,A human resources management system (HRMS) or ...,Item,Technology/Product/Service,The description indicates that a Human Resourc...,NaN
496,Central bank,"A central bank, reserve bank, national bank, o...",Non-item,organizations,The description provided defines Central Bank ...,NaN
497,Hydraulic engineering,Hydraulic engineering as a sub-discipline of c...,Item,Technology/Idea,Hydraulic engineering is a sub-discipline of c...,NaN
498,Director (military),"A director, also called an auxiliary predictor...",Item,Technology,The description of 'Director (military)' refer...,NaN
